# Example analysis of HCR c-fos experiment

this notebook provides a simple overview of shoaling index over time and a few sanity checks of standard HCR shoaling experiments.
it may also be useful as a starting point to undertand using the shoaling analysis code from Larsch & Baier 2018 and Kappel et al. 2022

This particular example looks at one of Deeksha's man2a2 experiments. Which has a biased ratio of hom vs. het animals and is therefore difficult to interpret.

Prerequisites:

**Raw data in one folder:**
- video of the experiment
- trajectory output file
- ROI file
- PL pair list
- camera calibration file

**Metadata in Excel file:**
- tab for AllExperiment list
- tab for AllAnimal list
1. column for genotype
2. Column for cohort (this can distinguish your stimuli, e.g. bout vs. invisble, for each animal)

see Y:\Deeksha\MetaData_DK.xlsx as an example Metadata file

**needs the jlsocialbehavior git repository and its dependencies**
- https://github.com/LarschLab/jlsocialbehavior

In [ ]:
#import socket
import os

RawDataDir = 'Y:\\Deeksha\\Behavior_mutant_screen\\RawData\\'
codeDir = 'D:\\Documents\\jlsocialbehavior'
ProcessingDir = RawDataDir
outputDir = 'Y:\\Deeksha\\'
MetaFile=outputDir+'MetaData_DK.xlsx'

print('RawDataDir = ' + RawDataDir)

os.chdir(codeDir)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import sys
import fnmatch

import math
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import glob
#import h5py
from datetime import datetime
import PythonMeta as PMA
from matplotlib.ticker import AutoLocator

import functions.matrixUtilities_joh as mu
import functions.notebookHelper as nh
import functions.metaTree as mt

import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
import functions.plotFunctions_joh as pf
pfp.paper()

In [ ]:
#read metadata
info=pd.read_excel(MetaFile,sheet_name='AllExp')
info

In [ ]:
#Select which experiment to process, e.g. based on the folderName
info=info[info.folder=='20250129_shoaling_man2a2']
info

In [ ]:
#separately read animal information
infoAn=pd.read_excel(MetaFile,sheet_name='AllAn')
infoAn.head()

In [ ]:
# Specify processing/analysis settings and save to csv file.

posPath=[]
PLPath=[]
expTime = []
birthDayAll=[]
anIDsAll=[]
camHeightAll=[]
#fileLenAll=[]

#camHeight=[105,180] #for arena up,dn
camHeight=[180,105] #for arena up,dn
for index,row in info.iterrows():
    startDir=RawDataDir+row.folder+'\\'
        
    posPathNow=glob.glob(startDir+'PositionTxt*.csv')
    if posPathNow:
        posPath.append(posPathNow[0])
        #fileLenAll.append(file_len(posPathNow[0]))
        #print(index,fileLenAll[-1])
        PLPath.append(glob.glob(startDir+'PL*.csv')[0])
        head, tail = os.path.split(posPath[-1])
        currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
        expTime.append(currTime)
        camHeightAll.append(camHeight[('_dn' in head)*1])
        
    else:
        posPath.append('')
        PLPath.append('')
        expTime.append('')
        camHeightAll.append('')    


    anNrs=row.anNr #Note that anNrs are 1 based!
    if ':' in anNrs:
        a,b=anNrs.split(sep=':')
        anNrs=np.arange(int(a),int(b)+1)
    else:
        anNrs=np.array(anNrs.split()).astype(int)
        
    anIDs=anNrs #-1 no more 0-based since using pandas merge to find animal numbers
    anIDsAll.extend(anIDs)

    bd=infoAn[infoAn.anNr.isin(anIDs)].bd.values
    #bd=infoAn.bd.values[anIDs-1] #a bit dirty to use anIDs directly here. Should merge
    birthDayAll.append(' '.join(list(bd)))

info.loc[:,'camHeight']=camHeightAll
info.loc[:,'aviPath']='default'
info.loc[:,'txtPath']=posPath
info.loc[:,'pairList']=PLPath
info.loc[:,'birthDayAll']=birthDayAll
info.loc[:,'epiDur'] = 5      # duration of individual episodes (default: 5 minutes)
info.loc[:,'episodes'] = 54   # number of episodes to process: -1 to load all episodes (default: -1)
info.loc[:,'inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
info.loc[:,'arenaDiameter_mm'] = 70 # arena diameter (default: 100 mm)
info.loc[:,'minShift'] = 60 # minimum number of seconds to shift for control IAD
info.loc[:,'episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info.loc[:,'recomputeAnimalSize'] = 1 # flag to compute animals size from avi file (takes time, default: 1)
info.loc[:,'SaveNeighborhoodMaps'] = 0 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info.loc[:,'computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info.loc[:,'ComputeBouts'] = 1 # flag to compute swim bout frequency (takes time, default: 1)
info.loc[:,'ComputeSync'] = 1 # flag to compute swim synchronization with stimulus (takes time, default: 1)
info.loc[:,'set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info.loc[:,'ProcessingDir']=ProcessingDir
info.loc[:,'outputDir']=outputDir
info.loc[:,'expTime']=expTime

csvFile=os.path.join(ProcessingDir,'processingSettings.csv')
info.to_csv(csvFile,encoding='utf-8')
info.tail()

In [ ]:
#Read and analyze the experiment data
# this generates an output csv file with shoaling indices for each animal for every 5 minute period.
# Also average speed, thigmotaxis, bout rate, animal size (based on video), and more esoteric indices

rereadData=1
if rereadData:
    def readExperiment(keepData=True):
        tmp=es.experiment_set(csvFile=csvFile,MissingOnly=False)
        if keepData:
            return tmp
        else:
            return 1

    expSet=readExperiment(keepData=True)

In [ ]:
#read results from the csv file into pandas df
#combine animal information from meta information

csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in info.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])

df=pd.DataFrame()
i=0
for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    tmp.animalSet=i
    tmp.animalIndex=tmp.animalIndex+((i)*35)
    tmp.animalIndex=np.array(anIDsAll)[tmp.animalIndex]
    df=pd.concat([df,tmp])
    i+=1
df['episode']=[x.strip().replace('_','') for x in df['episode']]
df=pd.merge(df,infoAn[['anNr','line','genotype','cohort']],left_on='animalIndex',right_on='anNr',how='left')
df=pd.merge(df,info[['date']],left_on='animalSet',right_on=info.index,how='left')

print('df shape',df.shape)
df.tail(10)

In [ ]:
df.info()

# Overview attraction over time

Plot shoaling index over time for each group.

In [ ]:
dfPlot=(df.groupby(['cohort','inDishTime','genotype']).si.agg(['mean','std'])
    .unstack()
    .stack(dropna=True)
    .reset_index())

dfPlot.head()


In [ ]:
fig, axes = plt.subplots(figsize=(6, 3))
g=sns.scatterplot(data=dfPlot,x='inDishTime',hue='cohort',y='mean')
plt.xlim([0,180])
plt.ylim([-.05,.4])
plt.xlabel('Time (Minutes)')
plt.ylabel('Attraction')
plt.title('Mean attraction, all animals - per stimulus group')
#plt.legend(title='Stimulus dot motion')
#plt.legend(labels=['Continuous motion','Bout motion','Continuous + loom','Bout + loom'],
#          title='Stimulus dot motion')

#new_title = 'Stimulus dot motion'
#g.legend_.set_title(new_title)
# replace labels
plt.legend(ncol=1,handletextpad=0,bbox_to_anchor=(1, 1),loc='upper left')
plt.axhline(0)

sns.despine()

#figPath=base+'ScreenTimeCourse.png'
#plt.savefig(figPath,bbox_inches='tight')


In [ ]:
dfSize=df.groupby(['animalIndex','genotype'])[['anSize','si']].mean().reset_index()


In [ ]:
sns.swarmplot(data=dfSize,x='genotype',y='anSize')

# Mean shoaling index over all stimuli per animal

In [ ]:
sns.set_palette('viridis',3)
co=sns.color_palette("viridis", 3)
idx=(df['inDishTime']<180) & (df['inDishTime']>120)
dfDR=df[idx]
dfEpiAn=dfDR.groupby(['cohort','animalIndex','line','genotype']).mean(numeric_only=True).reset_index()
#dfEpiAn['stim']=np.tile(['noStim','bout'],18)[:35]
#dfEpiAn.loc[0,'stim']='bout'

In [ ]:
dfDR.groupby(['animalIndex','cohort','line','genotype']).si.mean(numeric_only=True)

In [ ]:
dfDR.animalIndex.unique()

In [ ]:
dfEpiAn.cohort.unique()

In [ ]:
dfEpiAn.head(35)

In [ ]:
sns.stripplot(data=dfEpiAn,x='cohort',y='si',hue='genotype',zorder=-1, dodge=0.5)
#sns.pointplot(data=dfEpiAn,x='stim',y='si',hue='gtA',zorder=100,scale=0.2,palette=['gray'],join=False,dodge=0.5)
sns.pointplot(data=dfEpiAn,
              x='cohort',
              y='si',
              hue='genotype',
              join=False,
              color='k',
              dodge=0.5)
ax=plt.gca()
plt.axhline(0,ls=':',c='k')
#ax[0].legend()
h,l = ax.get_legend_handles_labels()
ax.legend(h[0:2], l[0:2],)   # <<<<<<<< This is where the magic happens
#ax.legend_.remove()

# Average swim behavior

In [ ]:
selDat=dfEpiAn[(dfEpiAn.cohort=='bout')]
selDat.loc[:,'group']=selDat.genotype
pf.plotGroupComparison(selDat,['h','m'],['si','avgSpeed_smooth','boutDur','thigmoIndex'],
                    labels=['Attraction','Swim speed (mm/s)','Bout duration (s)','Thigmotaxis (mm)'])
#plt.suptitle(info.stimulusProtocol[0])

In [ ]:
selDat=dfEpiAn[(dfEpiAn.cohort=='invisible')]
selDat.loc[:,'group']=selDat.genotype
pf.plotGroupComparison(selDat,['h','m'],['si','avgSpeed_smooth','boutDur','thigmoIndex'],
                    labels=['Attraction','Swim speed (mm/s)','Bout duration (s)','Thigmotaxis (mm)'])


# Below are some random bits to visualize speed/bouts by genotype

this is not a finished analysis

In [ ]:
allGt=df.groupby(['animalIndex']).genotype.first().values
allGtSortIndex=np.argsort(allGt)
allGt[allGtSortIndex]

In [ ]:
allSpeeds=np.array([x.ts.speed_smooth()[:30*60*9*5] for x in expSet.experiments[0].animals[:35]])
allSpeeds[np.isnan(allSpeeds)]=0

In [ ]:
fig, axes = plt.subplots(figsize=(8, 6))

s,l=50000,4000
plt.imshow(allSpeeds[allGtSortIndex,s:s+l],clim=[0,15], aspect='auto',cmap='plasma')
axes.set_yticks(np.arange(35));
axes.set_yticklabels(allGt[allGtSortIndex]);

In [ ]:
fig, axes = plt.subplots(figsize=(8, 6))
plt.plot(np.nanmean(allSpeeds,axis=0))
plt.axvline(30*60*9*5)

In [ ]:
header=pd.MultiIndex.from_arrays([allGt,df.animalIndex.unique()],names=['gt','animalIndex'])


In [ ]:
speedStack=pd.DataFrame(allSpeeds.T,columns=header).stack([0,1])
speedStack.name='speed'
speedStack=speedStack.reset_index()

In [ ]:
speedStack